In [22]:
import os, sys, string, time
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
import csv
from particle import Particle
# from matplotlib.patches import Rectangle

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print('Success')

Success


## Reading in single file

In [80]:

def Make_dataframes(file_loc):
    """
    Make dataframes given a .root file location.
    """
    interaction_truth_vars = ["nu_pdg", "ccnc", "npi0"]
    Vars_to_load = Variables.Final_variable_list + Variables.Truth_vars + interaction_truth_vars
    root_file = uproot3.open(file_loc)['nuselection/NeutrinoSelectionFilter']
    df_file = root_file.pandas.df(Vars_to_load, flatten=False)
    
    return df_file

Run = "run3"
df_to_test = Make_dataframes(f'../NuMI_MC/SLIMMED_neutrinoselection_filt_{Run}_overlay.root')
    
Functions.make_unique_ev_id(df_to_test)

print(f"Loaded {Run} files.")

df_to_test.keys()

Loaded run3 files.


Index(['run', 'sub', 'evt', 'nslice', 'n_pfps', 'n_tracks', 'n_showers',
       'swtrig_pre', 'swtrig_post', 'trk_sce_start_x_v', 'trk_sce_start_y_v',
       'trk_sce_start_z_v', 'trk_sce_end_x_v', 'trk_sce_end_y_v',
       'trk_sce_end_z_v', 'shr_theta_v', 'shr_phi_v', 'shr_px_v', 'shr_py_v',
       'shr_pz_v', 'shrclusdir0', 'shrclusdir1', 'shrclusdir2',
       'shr_energy_tot', 'trk_theta_v', 'trk_phi_v', 'trk_dir_x_v',
       'trk_dir_y_v', 'trk_dir_z_v', 'trk_energy', 'trk_energy_hits_tot',
       'trk_energy_tot', 'trk_score_v', 'trk_calo_energy_u_v', 'trk_end_x_v',
       'trk_chipr_best', 'pfnplanehits_U', 'pfnplanehits_V', 'pfnplanehits_Y',
       'NeutrinoEnergy2', 'SliceCaloEnergy2', 'nu_flashmatch_score',
       'contained_sps_ratio', 'flash_time', 'contained_fraction', 'trk_score',
       'crtveto', 'shr_tkfit_dedx_U', 'shr_tkfit_dedx_V', 'shr_tkfit_dedx_Y',
       'shr_tkfit_dedx_max', 'shr_tkfit_2cm_dedx_Y', 'shr_chipr',
       'trk_bragg_p', 'trk_bragg_p_v', 'trk_chipr'

In [88]:
#Set these
Run = 16219
SubRun = 10
Event = 513

converted_rse = str(Run) + "_" + str(SubRun) + "_" + str(Event)

print(converted_rse)

isolated_event = df_to_test.loc[(df_to_test['rse_id'].isin([converted_rse]))]

# print(isolated_event)


16219_10_513


In [89]:
print("Truth pdgs are: ")

print(isolated_event["mc_pdg"])

entry = isolated_event.index[0]
truth_particles = []

print("Particles are: ")
for pdg in isolated_event["mc_pdg"][entry]:
    print(Particle.from_pdgid(int(pdg)))
    truth_particles.append(str(Particle.from_pdgid(int(pdg))))

Truth pdgs are: 
entry
587538    [14, 2112, 2212, -211, 2212]
Name: mc_pdg, dtype: object
Particles are: 
nu(mu)
n
p
pi-
p


In [90]:
print("Interaction type is: ")
nu_pdg = isolated_event["nu_pdg"][entry]
ccnc = isolated_event["ccnc"][entry]
npi0 = isolated_event["npi0"][entry]
print("Code: "+str(isolated_event["nu_pdg"][entry]))
print("ccnc is: " + str(ccnc))

r"$\nu_{\mu}$ CC $0\pi^{0}$", r"$\nu_{\mu}$ NC $n\pi^{0}$", r"$\nu_{\mu}$ NC $0\pi^{0}$", r"$\nu_{e}$"

if (nu_pdg==14 or nu_pdg==-14) and ccnc==0 and npi0>=1: interaction_type = r"$\nu_{\mu}$ CC $n\pi^{0}$"
elif (nu_pdg==14 or nu_pdg==-14) and ccnc==0 and npi0>=1: interaction_type = r"$\nu_{\mu}$ CC $0\pi^{0}$"
elif (nu_pdg==14 or nu_pdg==-14) and ccnc==1 and npi0>=1: interaction_type = r"$\nu_{\mu}$ NC $n\pi^{0}$"
elif (nu_pdg==14 or nu_pdg==-14) and ccnc==1 and npi0==0: interaction_type = r"$\nu_{\mu}$ NC $0\pi^{0}$"
elif (nu_pdg==12 or nu_pdg==-12): interaction_type = r"$\nu_{e}$"

print(interaction_type)

Interaction type is: 
Code: 14
ccnc is: 1
$\nu_{\mu}$ NC $0\pi^{0}$


In [91]:
print("Total Energies are: ")

for i, Energy in enumerate(isolated_event["mc_E"][entry]):
    print(truth_particles[i]+": "+str(Energy)+ " GeV")

Total Energies are: 
nu(mu): 0.75277126 GeV
n: 0.97791857 GeV
p: 0.98876715 GeV
pi-: 0.3183972 GeV
p: 0.9394203 GeV


In [100]:
magnitude_momenta = []
direction_particle = []

def Get_magnitude_mom(px, py, pz):
    """
    Given individual components of momenta, returns the magnitude of momentum.
    """
    squared_sum = px**2 + py**2 + pz**2
    magnitude = np.sqrt(squared_sum)
    return magnitude

def Get_direction(mom_1, mom_2, direction):
    """
    Provide 2 lists of px, py, pz.
    Returns their angles wrt \"direction\" vector provided.
    Also returns the opening angle between the two momenta lists. 
    """
    dot_1 = direction[0]*mom_1[0] + direction[1]*mom_1[1] + direction[2]*mom_1[2] # a dot b
    dot_2 = direction[0]*mom_2[0] + direction[1]*mom_2[1] + direction[2]*mom_2[2] # a dot b
    mag_1 = np.sqrt(mom_1[0]**2 + mom_1[1]**2 + mom_1[2]**2)
    mag_2 = np.sqrt(mom_2[0]**2 + mom_2[1]**2 + mom_2[2]**2)
    mag_dir = np.sqrt(direction[0]**2 + direction[1]**2 + direction[2]**2)

    p_sum = [mom_1[0]+mom_2[0], mom_1[1]+mom_2[1], mom_1[2]+mom_2[2]]
    p_sum_dot_dir = p_sum[0]*direction[0] + p_sum[1]*direction[1] + p_sum[2]*direction[2]
    p_sum_mag = np.sqrt((mom_1[0]+mom_2[0])**2+(mom_1[1]+mom_2[1])**2+(mom_1[2]+mom_2[2])**2) # magnitude 1 plus 2

    direction_1 = np.arccos(dot_1/(mag_1*mag_dir))
    direction_2 = np.arccos(dot_2/(mag_2*mag_dir))
    combined_angle = np.arccos(p_sum_dot_dir/(p_sum_mag*mag_dir))
        
    return direction_1, direction_2, combined_angle

def Convert_radians_to_deg(radians):
    """
    Input radians, returns degrees
    """
    degrees = radians*180/np.pi
    return degrees
    
    
print("Magnitude of momenta are: ")
for i, Energy in enumerate(isolated_event["mc_E"][entry]):
    px = isolated_event["mc_px"][entry][i]
    py = isolated_event["mc_py"][entry][i]
    pz = isolated_event["mc_pz"][entry][i]
    
    mag_p = Get_magnitude_mom(px, py, pz)
    
    print(truth_particles[i]+": "+str(mag_p)+ " GeV")
    
interest_p1 = 3
interest_p2 = 4
mom_1 = [isolated_event["mc_px"][entry][interest_p1], isolated_event["mc_py"][entry][interest_p1], isolated_event["mc_pz"][entry][interest_p1]]
mom_2 = [isolated_event["mc_px"][entry][interest_p2], isolated_event["mc_py"][entry][interest_p2], isolated_event["mc_pz"][entry][interest_p2]]


Magnitude of momenta are: 
nu(mu): 0.7527712686908294 GeV
n: 0.271185701518239 GeV
p: 0.31193919295009015 GeV
pi-: 0.2861764362666339 GeV
p: 0.04643371392024028 GeV


In [101]:
z_dir = [0,0,1]
av_HNL_dir = [0.324,0.738,-0.591]

uboone_angle_1, uboone_angle_2, uboone_combined_angle = Get_direction(mom_1, mom_2, z_dir)
absorber_angle_1, absorber_angle_2, absorber_combined_angle = Get_direction(mom_1, mom_2, av_HNL_dir)

print("Angles wrt. z dir. are: ")

print(truth_particles[interest_p1]+": "+str(uboone_angle_1)+ " radians")
print(truth_particles[interest_p2]+": "+str(uboone_angle_2)+ " radians")
print("Combined: " + str(uboone_combined_angle)+ " radians")

print(truth_particles[interest_p1]+": "+str(Convert_radians_to_deg(uboone_angle_1))+ " degrees")
print(truth_particles[interest_p2]+": "+str(Convert_radians_to_deg(uboone_angle_2))+ " degrees")
print("Combined: " + str(Convert_radians_to_deg(uboone_combined_angle))+ " degrees")

print()
print("Angles wrt. absorber are: ")

print(truth_particles[interest_p1]+": "+str(absorber_angle_1)+ " radians")
print(truth_particles[interest_p2]+": "+str(absorber_angle_2)+ " radians")
print("Combined: " + str(absorber_combined_angle)+ " radians")

print(truth_particles[interest_p1]+": "+str(Convert_radians_to_deg(absorber_angle_1))+ " degrees")
print(truth_particles[interest_p2]+": "+str(Convert_radians_to_deg(absorber_angle_2))+ " degrees")
print("Combined: " + str(Convert_radians_to_deg(absorber_combined_angle))+ " degrees")


Angles wrt. z dir. are: 
pi-: 0.9457082196277758 radians
p: 1.2108968258427284 radians
Combined: 0.937686304076121 radians
pi-: 54.185089635502685 degrees
p: 69.3792775465762 degrees
Combined: 53.72546773078249 degrees

Angles wrt. absorber are: 
pi-: 2.087570347896273 radians
p: 2.847540483303912 radians
Combined: 2.2119938298920023 radians
pi-: 119.60897037111341 degrees
p: 163.1520516859568 degrees
Combined: 126.73791076179069 degrees


# Finished code